In [17]:
import os

import joblib
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1. / 255)

valid_datagen = ImageDataGenerator(rescale=1. / 255)

In [19]:
SAVE_DIR = "/Users/Bilal/PycharmProjects/LearningAI/models/assignment_7/task_3"
MODEL_PATH = f"{SAVE_DIR}/food_model.keras"
HISTORY_PATH = f"{SAVE_DIR}/history.pkl"

In [20]:
train_dir = "/Users/Bilal/PycharmProjects/LearningAI/Files/assignment_7/task_3/10_food_classes_all_data/train"
test_dir = "/Users/Bilal/PycharmProjects/LearningAI/Files/assignment_7/task_3/10_food_classes_all_data/test"

train_data = train_datagen.flow_from_directory(
    train_dir,
    color_mode="rgb",
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
)

test_data = valid_datagen.flow_from_directory(
    test_dir,
    color_mode="rgb",
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
)



Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [21]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomFlip(mode='horizontal'),

])


In [22]:
model = tf.keras.Sequential([

    tf.keras.layers.Input(shape=(224, 224, 3)),
    data_augmentation,

    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation="softmax")
])


In [23]:
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [25]:


if os.path.exists(MODEL_PATH) and os.path.exists(HISTORY_PATH):
    print("Loading Already Present Model")
    model = tf.keras.models.load_model(MODEL_PATH)
    history = joblib.load(HISTORY_PATH)
    #evaulation
    pd.DataFrame(history).plot()

else:
    print("Making a new Model")

    history = model.fit(train_data, epochs=15, validation_data=test_data)
    model.save(MODEL_PATH)
    #savng history
    joblib.dump(history.history, HISTORY_PATH)

    #evaluation
    pd.DataFrame(history.history).plot()

Making a new Model
Epoch 1/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 723s 3s/step - accuracy: 0.2712 - loss: 2.0751 - val_accuracy: 0.3440 - val_loss: 1.9510
Epoch 2/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 718s 3s/step - accuracy: 0.2676 - loss: 2.0637 - val_accuracy: 0.3540 - val_loss: 1.9664
Epoch 3/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 1378s 6s/step - accuracy: 0.2817 - loss: 2.0570 - val_accuracy: 0.3596 - val_loss: 1.9707
Epoch 4/15
 30/235 ━━━━━━━━━━━━━━━━━━━━ 10:29 3s/step - accuracy: 0.3066 - loss: 2.0350

KeyboardInterrupt: 